# LSTM Stock Predictor Using Closing Prices

In this notebook, you will build and train a custom LSTM RNN that uses a 10 day window of Bitcoin closing prices to predict the 11th day closing price. 

You will need to:

1. Prepare the data for training and testing
2. Build and train a custom LSTM RNN
3. Evaluate the performance of the model

## Data Preparation

In this section, you will need to prepare the training and testing data for the model. The model will use a rolling 10 day window to predict the 11th day closing price.

You will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model. Note: The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [ ]:
import numpy as np
import pandas as pd
import hvplot.pandas
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
# Set the random seed for reproducibility
# Note: This is for the homework solution, but it is good practice to comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [ ]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('./data/btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df.head()

In [ ]:
# Load the historical closing prices for bitcoin
df2 = pd.read_csv('./data/btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2.tail()

In [ ]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

In [ ]:
df.head()

In [ ]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i : (i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [ ]:
# Predict Closing Prices using a 10 day window of previous closing prices
# Try a window size anywhere from 1 to 10 and see how the model performance changes
window_size = 3

# Column index 1 is the `Close` column

feature_column = 1
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)
print(X[0:2])
X.shape

In [ ]:
# Use 70% of the data for training and the remaineder for testing

split = int(0.7 * len(X))
X_train = X[: split - 1]
X_test = X[split:]
y_train = y[: split - 1]
y_test = y[split:]

print(len(y_train)/(len(y_train)+len(y_test)))

In [ ]:
# Use MinMaxScaler to scale the data between 0 and 1. 
# YOUR CODE HERE
from sklearn.preprocessing import StandardScaler
scaler = MinMaxScaler()
scaler.fit(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
scaler.fit(y)
y_train = scaler.transform(y_train)
y_test = scaler.transform(y_test)

In [ ]:
# Reshape the features for the model
# YOUR CODE HERE!
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
X_test[:2]

---

## Build and Train the LSTM RNN

In this section, you will design a custom LSTM RNN and fit (train) it using the training data.

You will need to:
1. Define the model architecture
2. Compile the model
3. Fit the model to the training data

### Hints:
You will want to use the same model architecture and random seed for both notebooks. This is necessary to accurately compare the performance of the FNG model vs the closing price model. 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
# Build the LSTM model. 
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 
# YOUR CODE HERE!
model = Sequential()

number_units = 7
dropout_fraction = .2

# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))
# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))
# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))
# Output layer
model.add(Dense(1))

In [ ]:
# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error", metrics="mean_absolute_percentage_error")

In [ ]:
# Summarize the model
model.summary()

In [ ]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended
# YOUR CODE HERE!
model.fit(X_train, y_train, epochs=20, shuffle=False, batch_size=1, verbose=True)

Epoch 1/20


---

## Model Performance

In this section, you will evaluate the model using the test data. 

You will need to:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

### Hints
Remember to apply the `inverse_transform` function to the predicted and y_test values to recover the actual closing prices.

In [18]:
# Evaluate the model
model.evaluate(X_test, y_test, verbose=True)

6/6 [==============================] - 0s 3ms/step - loss: 0.2008 - mean_absolute_percentage_error: 36.9330


[0.20083752274513245, 36.93299865722656]

In [140]:
# Make some predictions
# YOUR CODE HERE!
predicted = model.predict(X_test)
predicted.shape

(162, 1)

In [21]:
# Recover the original prices instead of the scaled version
predicted_prices = scaler.inverse_transform(predicted)
real_prices = scaler.inverse_transform(y_test.reshape(-1, 1))

NameError: name 'predicted' is not defined

In [ ]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
})
stocks.head()

In [ ]:
# Plot the real vs predicted values as a line chart
stocks.hvplot(y=['Real', 'Predicted'])

In [82]:
windows_l=[]
windows_a=[]
for win in range(1,11):
    print(f'window {win}')
    nodes_l=[]
    nodes_a=[]
    for node in range(1,32,3):
        epo_l=[]
        epo_a=[]
        for epo in range(10,22,2):
            print(f'node {node}')
            window_size = win

            feature_column = 1
            target_column = 1
            X, y = window_data(df, window_size, feature_column, target_column)
            split = int(0.7 * len(X))
            X_train = X[: split - 1]
            X_test = X[split:]
            y_train = y[: split - 1]
            y_test = y[split:]
            scaler = MinMaxScaler()
            scaler.fit(X)
            X_train = scaler.transform(X_train)
            X_test = scaler.transform(X_test)
            scaler.fit(y)
            y_train = scaler.transform(y_train)
            y_test = scaler.transform(y_test)
            X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
            X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
            model = Sequential()

            number_units = node
            dropout_fraction = .2

            # Layer 1
            model.add(LSTM(
                units=number_units,
                return_sequences=True,
                input_shape=(X_train.shape[1], 1))
                )
            model.add(Dropout(dropout_fraction))
            # Layer 2
            model.add(LSTM(units=number_units, return_sequences=True))
            model.add(Dropout(dropout_fraction))
            # Layer 3
            # model.add(LSTM(units=number_units))
            # model.add(Dropout(dropout_fraction))
            # Output layer
            model.add(Dense(1))
            model.compile(optimizer="adam", loss="mean_squared_error", metrics="mean_absolute_percentage_error")
            model.fit(X_train, y_train, epochs=epo, shuffle=False, batch_size=1, verbose=False)
            eval=model.evaluate(X_test, y_test)
            epo_l.append(eval[0])
            epo_a.append(eval[1])
        nodes_l.append(epo_l)
        nodes_a.append(epo_a)
        print('node done')
    print('window done')
    windows_l.append(nodes_l)
    windows_a.append(nodes_a)
print('done')

window 1
node 1
6/6 [==============================] - 0s 3ms/step - loss: 0.0466 - mean_absolute_percentage_error: 46.1127
node 1
6/6 [==============================] - 0s 2ms/step - loss: 0.0228 - mean_absolute_percentage_error: 34.5832
node 1
6/6 [==============================] - 0s 3ms/step - loss: 0.0209 - mean_absolute_percentage_error: 33.3968
node 1
6/6 [==============================] - 0s 2ms/step - loss: 0.0249 - mean_absolute_percentage_error: 37.0478
node 1
6/6 [==============================] - 0s 2ms/step - loss: 0.0217 - mean_absolute_percentage_error: 36.1288
node 1
6/6 [==============================] - 0s 2ms/step - loss: 0.0295 - mean_absolute_percentage_error: 32.3969
node done
node 4
6/6 [==============================] - 0s 2ms/step - loss: 0.0166 - mean_absolute_percentage_error: 26.2553
node 4
6/6 [==============================] - 0s 3ms/step - loss: 0.0181 - mean_absolute_percentage_error: 27.7401
node 4
6/6 [==============================] - 0s 2ms/step - l

In [114]:
ep12=[]

ep12l=[]

for win in range(10):
    ep12nl=[]
    ep12n=[]
    for nod in range(11):
        ep12nl.append(windows_l[win][nod][4])
        ep12n.append(windows_a[win][nod][4])

    ep12.append(ep12n)
    ep12l.append(ep12nl)

dfep12l = pd.DataFrame(ep12l, columns = ['nod1','nod3','nod7','nod10','nod13','nod16','nod19','nod22','nod25','nod28','nod31']) 
dfep12l.index = ['win1','win2','win3','win4','win5','win6','win7','win8','win9','win10']

dfep12 = pd.DataFrame(ep12, columns = ['nod1','nod3','nod7','nod10','nod13','nod16','nod19','nod22','nod25','nod28','nod31']) 
dfep12.index =['win1','win2','win3','win4','win5','win6','win7','win8','win9','win10']

dfep12l.head()

,nod1,nod3,nod7,nod10,nod13,nod16,nod19,nod22,nod25,nod28,nod31
win1,0.021739,0.008683,0.008243,0.009906,0.008838,0.012196,0.016745,0.013794,0.018368,0.019137,0.017744
win2,0.025771,0.013240,0.014139,0.016919,0.016161,0.021553,0.023156,0.021987,0.022696,0.025160,0.025390
win3,0.028879,0.023624,0.017419,0.018674,0.022717,0.022078,0.023776,0.026027,0.025091,0.025821,0.028014
win4,0.097603,0.018167,0.021415,0.022210,0.025024,0.023824,0.025223,0.026741,0.027434,0.026998,0.027751
win5,0.040523,0.025445,0.023704,0.026819,0.026147,0.028408,0.029168,0.029874,0.031182,0.032154,0.032259


In [115]:
import hvplot.pandas
dfep12l.hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [116]:
dfep12.hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [117]:
win4=[]

win4l=[]

for nod in range(11):
    win4nl=[]
    win4n=[]
    for ep in range(6):
        win4nl.append(windows_l[2][nod][ep])
        win4n.append(windows_a[2][nod][ep])

    win4.append(win4n)

    win4l.append(win4nl)

dfwin4l = pd.DataFrame(win4l, columns = ['10 epoch', '12 epoch','14 epoch','16 epoch','18 epoch','20 epoch']) 
dfwin4l.index = ['nod1','nod3','nod7','nod10','nod13','nod16','nod19','nod22','nod25','nod28','nod31']

dfwin4 = pd.DataFrame(win4, columns = ['10 epoch', '12 epoch','14 epoch','16 epoch','18 epoch','20 epoch']) 
dfwin4.index =['nod1','nod3','nod7','nod10','nod13','nod16','nod19','nod22','nod25','nod28','nod31']

dfwin4l.head()

,10 epoch,12 epoch,14 epoch,16 epoch,18 epoch,20 epoch
nod1,0.079800,0.040138,0.039473,0.033719,0.028879,0.025760
nod3,0.023923,0.023761,0.024416,0.018673,0.023624,0.022463
nod7,0.026970,0.020713,0.018450,0.017674,0.017419,0.018556
nod10,0.021710,0.020210,0.020215,0.022106,0.018674,0.020581
nod13,0.023376,0.025845,0.021054,0.024572,0.022717,0.019663


In [118]:
dfwin4.hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [119]:
dfwin4l.hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [120]:
nod3=[]

nod3l=[]

for win in range(10):
    nod3nl=[]
    nod3n=[]
    for ep in range(6):
        nod3nl.append(windows_l[win][2][ep])
        nod3n.append(windows_a[win][2][ep])

    nod3.append(nod3n)
    nod3l.append(nod3nl)

dfnod3l = pd.DataFrame(nod3l, columns = ['10 epoch', '12 epoch','14 epoch','16 epoch','18 epoch','20 epoch']) 
dfnod3l.index = ['win1','win2','win3','win4','win5','win6','win7','win8','win9','win10']

dfnod3 = pd.DataFrame(nod3, columns = ['10 epoch', '12 epoch','14 epoch','16 epoch','18 epoch','20 epoch']) 
dfnod3.index =['win1','win2','win3','win4','win5','win6','win7','win8','win9','win10']

dfnod3l.head()

,10 epoch,12 epoch,14 epoch,16 epoch,18 epoch,20 epoch
win1,0.015007,0.010035,0.008099,0.007165,0.008243,0.010869
win2,0.017363,0.016229,0.015707,0.011488,0.014139,0.013938
win3,0.026970,0.020713,0.018450,0.017674,0.017419,0.018556
win4,0.029278,0.026693,0.021091,0.021267,0.021415,0.021793
win5,0.029806,0.028865,0.025529,0.025715,0.023704,0.021409


In [121]:
dfnod3l.hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [122]:
dfnod3.hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)